In [ ]:

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
import numpy as np

train_dir = r'D:\archive\train'  
val_dir = 'dataset/val'     
test_dir = 'dataset/test'    

IMG_SIZE = (128, 128)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))
base_model.trainable = False  

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

EPOCHS = 5 
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

model.save("fruit_classifier_mobilenetv2.h5")

def predict_image(img_path, confidence_threshold=0.6):
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    class_labels = {v:k for k,v in train_generator.class_indices.items()}
    confidence = prediction[0][class_idx]
    
    if confidence < confidence_threshold:
        return "Not a fruit"
    else:
        return f"{class_labels[class_idx]} (Confidence: {confidence:.2f})"

test_image_path = 'test_images/sample.jpg' 
result = predict_image(test_image_path)
print(result)


In [ ]:


import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
import numpy as np
import tkinter as tk
from tkinter import filedialog
import os

train_dir = r'D:\archive\train'  
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))
base_model.trainable = False  

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

EPOCHS = 5
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

model.save("fruit_classifier_mobilenetv2.h5")

def predict_image(img_path, confidence_threshold=0.6):
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    class_labels = {v:k for k,v in train_generator.class_indices.items()}
    confidence = prediction[0][class_idx]
    
    if confidence < confidence_threshold:
        return "Not a fruit"
    else:
        return f"{class_labels[class_idx]} (Confidence: {confidence:.2f})"

def select_and_predict():
    file_path = filedialog.askopenfilename()
    if file_path:
        result = predict_image(file_path)
        result_label.config(text=f"Prediction: {result}")


root = tk.Tk()
root.title("Fruit/Non-Fruit Detector")

btn = tk.Button(root, text="Select Image", command=select_and_predict)
btn.pack(pady=20)

result_label = tk.Label(root, text="Prediction: ", font=("Arial", 14))
result_label.pack(pady=20)

root.mainloop()
